# Introducción

Armamos un dataframe con los datos importantes.

IMPORTANTE: para que funcione hay que ubicar los siguientes csv en la misma carpeta que este notebook:

* properati-AR-2016-06-01-properties-sell.csv
* properati-AR-2017-01-01-properties-sell.csv
* properati-AR-2017-08-01-properties-sell-six_months.csv
* properati-AR-2017-10-01-properties-sell.csv


## Lectura y agrupamiento de los datos

Primero importamos las librerias necesarias para poder realizar la exploración de datos, para eso importamos pandas, numpy, matplotlib y demas librerías, también lo configuramos para que los gráficos se vean bien en pantalla.

In [1]:
import datetime as datetime
import pandas as pd
import numpy as np
import math

#### Preparación de los dataframes

Al tener diferentes archivos desde los cuales extraemos datos, los cuales en conjunto abarcan un período de 2 años de publicaciones, debemos unificarlos en un solo dataframe, teniendo cuidado de que no se repitan datos. A su vez, nos quedamos solamente con los registros de CABA y G.B.A.

In [2]:
df7 = pd.read_csv('properati-AR-2016-06-01-properties-sell.csv', low_memory=False)
fecha = (df7['created_on']>= '2015-12-01') & (df7['created_on']<= '2016-06-30')
caba = df7.place_with_parent_names.str.contains('Capital')
gba = df7.place_with_parent_names.str.contains('G.B.A')

df7 = df7.loc[fecha & (caba | gba),:]

In [3]:
df8 = pd.read_csv('properati-AR-2017-01-01-properties-sell.csv', low_memory=False)
fecha = (df8['created_on']>= '2016-07-01') & (df8['created_on']<= '2017-01-31')
caba = df8.place_with_parent_names.str.contains('Capital')
gba = df8.place_with_parent_names.str.contains('G.B.A')

df8 = df8.loc[fecha & (caba | gba),:]

In [4]:
df9 = pd.read_csv('properati-AR-2017-08-01-properties-sell-six_months.csv', low_memory=False)
fecha = (df9['created_on']>= '2017-02-01') & (df9['created_on']<= '2017-07-31')
caba = df9.place_with_parent_names.str.contains('Capital')
gba = df9.place_with_parent_names.str.contains('G.B.A')

df9 = df9.loc[fecha & (caba | gba),:]

In [5]:
df10 = pd.read_csv('properati-AR-2017-10-01-properties-sell.csv', low_memory=False)
fecha = (df10['created_on']> '2017-07-31')
caba = df10.place_with_parent_names.str.contains('Capital')
gba = df10.place_with_parent_names.str.contains('G.B.A')

df10 = df10.loc[fecha & (caba | gba),:]

Ahora ya tenemos 4 dataframes y lo siguiente a hacer es unificarlos en un único dataframe que los contenga a todos.

Vemos que los campos que comparten son: 'created_on','property_type','place_name','place_with_parent_names','geonames_id','lat-lon','lat','lon','price','currency','price_aprox_local_currency','price_aprox_usd','price_usd_per_m2','floor','rooms','expenses'.

Veamos que sucede en el resto de las columnas, y si hay alguna forma de homogeneizar la información.
Empecemos agrupando los dataframes.

In [6]:
df7a10 = pd.concat([df7,df8,df9,df10])

Sacaremos información redundate o que no aporta al análisis, como datos internos que guarde Properati por diversas razones que no conocemos.
* '**id**': No nos aporta información.
* '**operation**': Todas las operaciones son de venta.
* '**geonames_id**': Es un id interno de Properati.
* '**place_with_parent_names**': Todo el análisis es de Argentina, y en particular solo nos quedaremos con las propiedades de CABA/GBA. El dato del estado al que pertenece esta en 'state_name', y el barrio en 'place_name' por lo que es redundante.
* '**lat-lon**': Es un campo que concatena los datos de los campos 'lat' y 'lon', por lo que es redundante.
* '**properati_url**': No nos aporta información.
* '**image_thumbnail**': No nos aporta información.



In [7]:
df7a10 = df7a10.drop(df7a10[['id','operation','place_with_parent_names', 'geonames_id','properati_url','image_thumbnail','country_name', 'title', 'description']], axis = 1)

In [8]:
df7a10.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 191387 entries, 1 to 196718
Data columns (total 18 columns):
created_on                    191387 non-null object
property_type                 191387 non-null object
place_name                    191333 non-null object
state_name                    191387 non-null object
lat-lon                       127252 non-null object
lat                           127252 non-null float64
lon                           127252 non-null float64
price                         174886 non-null float64
currency                      172953 non-null object
price_aprox_local_currency    174886 non-null float64
price_aprox_usd               174886 non-null float64
surface_total_in_m2           144233 non-null float64
surface_covered_in_m2         171452 non-null float64
price_usd_per_m2              127466 non-null float64
price_per_m2                  156866 non-null float64
floor                         17330 non-null float64
rooms                         99

Para que queden las mismas columnas, en df7a10 tenemos que unificar '**surface_total_in_m2**' con '**surface_covered_in_m2**', y '**price_usd_per_m2**' con '**price_per_m2**'. 
Limpiemos el dataframe de los registros con datos de superficie invalidos, ya que es un dato clave y sin este dato no podemos determinar el valor de una propiedad.

In [9]:
def obtener_superficie_final(covered, total):
    if (covered == total): return total
    if ((math.isnan(covered) or covered == 0) and (math.isnan(total) or total == 0)): return 0
    if (math.isnan(total) or total == 0): return covered
    if (math.isnan(covered) or covered == 0): return total
    return (covered + total)/2

In [10]:
df7a10['surface_in_m2'] = df7a10[['surface_covered_in_m2','surface_total_in_m2']].apply(lambda x: obtener_superficie_final(x[0],x[1]), axis = 1)
df7a10 = df7a10.drop(df7a10[['surface_covered_in_m2','surface_total_in_m2']],axis=1)

Ahora unifiquemos el precio por m2, ya que el df7a10 tiene dos columnas referidas a los mismo: "price_usd_per_m2" y "price_usd_per_m2". Vamos a sacar los que sean NaN, al igual que hicimos con las superficies.

In [11]:
price_usd_per_m2_is_invalid = np.isnan(df7a10['price_usd_per_m2'])
price_per_m2_is_invalid = np.isnan(df7a10['price_per_m2'])
invalid_en_simultaneo = price_usd_per_m2_is_invalid & price_per_m2_is_invalid
df7a10 = df7a10[np.logical_not(invalid_en_simultaneo)]

ambos_iguales = df7a10['price_per_m2'] == df7a10['price_usd_per_m2']
ambos_iguales.value_counts()

False    126513
True      36316
dtype: int64

Concatenemos todas las publicaciones en un unico dataframe llamado **properties**.

In [12]:
columns7a10 = df7a10.columns.tolist().sort()

In [13]:
properties = df7a10

## Limpieza de los datos

Analicemos el dataframe

In [14]:
properties.info()
properties.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162829 entries, 1 to 196718
Data columns (total 17 columns):
created_on                    162829 non-null object
property_type                 162829 non-null object
place_name                    162775 non-null object
state_name                    162829 non-null object
lat-lon                       107725 non-null object
lat                           107725 non-null float64
lon                           107725 non-null float64
price                         162829 non-null float64
currency                      162825 non-null object
price_aprox_local_currency    162829 non-null float64
price_aprox_usd               162829 non-null float64
price_usd_per_m2              127466 non-null float64
price_per_m2                  156866 non-null float64
floor                         16351 non-null float64
rooms                         87160 non-null float64
expenses                      23464 non-null float64
surface_in_m2                 1628

,lat,lon,price,price_aprox_local_currency,price_aprox_usd,price_usd_per_m2,price_per_m2,floor,rooms,expenses,surface_in_m2
count,107725.000000,107725.000000,1.628290e+05,1.628290e+05,1.628290e+05,1.274660e+05,1.568660e+05,16351.000000,87160.000000,2.346400e+04,1.628290e+05
mean,-34.572762,-58.514387,3.581461e+05,4.622681e+06,2.721524e+05,2.134135e+03,4.665813e+03,19.512813,2.968369,4.239002e+03,2.106709e+02
std,0.713836,0.680066,8.221022e+05,6.509589e+06,3.835435e+05,4.176744e+03,2.164063e+04,131.562859,1.526454,9.248221e+04,8.513424e+03
min,-41.167596,-122.419415,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.275958e-02,1.000000,1.000000,1.000000e+00,1.000000e+00
25%,-34.625205,-58.579658,1.120000e+05,1.817865e+06,1.080000e+05,1.233766e+03,1.567812e+03,2.000000,2.000000,1.100000e+03,5.000000e+01
50%,-34.592678,-58.473744,1.820000e+05,2.925897e+06,1.720000e+05,1.878049e+03,2.162162e+03,3.000000,3.000000,2.082000e+03,8.550000e+01
75%,-34.528291,-58.414348,3.463350e+05,5.269605e+06,3.100000e+05,2.577320e+03,3.003226e+03,6.000000,4.000000,4.100000e+03,1.910000e+02
max,37.774929,3.379206,1.234568e+08,8.212711e+08,5.000000e+07,1.271027e+06,4.000000e+06,7071.000000,32.000000,1.000150e+07,3.243242e+06


El dataframe original tiene 162829 publicaciones, al final de la limpieza veremos cuántas nos sirven para el análisis.

Verifiquemos que las propiedades sin 'price' (NaN) no tienen los datos suficientes para calcular dicho campo (superficie + precio por m2).

In [15]:
properties[np.isnan(properties.price)][['price','currency','price_aprox_local_currency','price_aprox_usd','price_usd_per_m2','lat','lon']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 7 columns):
price                         0 non-null float64
currency                      0 non-null object
price_aprox_local_currency    0 non-null float64
price_aprox_usd               0 non-null float64
price_usd_per_m2              0 non-null float64
lat                           0 non-null float64
lon                           0 non-null float64
dtypes: float64(6), object(1)
memory usage: 0.0+ bytes


Se ve que para todo registro que no tiene price, tampoco hay datos suficientes como para calcularlo. Nos quedamos solamente con los registros que tienen el 'price' por m2 completo

In [16]:
properties.dropna(subset=['price'], inplace=True)
properties.info()
properties.dropna(subset=['lat'], inplace=True)
properties.dropna(subset=['lon'], inplace=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162829 entries, 1 to 196718
Data columns (total 17 columns):
created_on                    162829 non-null object
property_type                 162829 non-null object
place_name                    162775 non-null object
state_name                    162829 non-null object
lat-lon                       107725 non-null object
lat                           107725 non-null float64
lon                           107725 non-null float64
price                         162829 non-null float64
currency                      162825 non-null object
price_aprox_local_currency    162829 non-null float64
price_aprox_usd               162829 non-null float64
price_usd_per_m2              127466 non-null float64
price_per_m2                  156866 non-null float64
floor                         16351 non-null float64
rooms                         87160 non-null float64
expenses                      23464 non-null float64
surface_in_m2                 1628

Limpiemos el dataframe de los registros sin datos de la superficie, ya que es un dato clave y sin este dato no podemos determinar el valor de una propiedad.

In [17]:
surface_invalid = np.isnan(properties['surface_in_m2'])
properties = properties[np.logical_not(surface_invalid)]
properties.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107725 entries, 16 to 196715
Data columns (total 17 columns):
created_on                    107725 non-null object
property_type                 107725 non-null object
place_name                    107671 non-null object
state_name                    107725 non-null object
lat-lon                       107725 non-null object
lat                           107725 non-null float64
lon                           107725 non-null float64
price                         107725 non-null float64
currency                      107721 non-null object
price_aprox_local_currency    107725 non-null float64
price_aprox_usd               107725 non-null float64
price_usd_per_m2              86391 non-null float64
price_per_m2                  103876 non-null float64
floor                         12034 non-null float64
rooms                         70060 non-null float64
expenses                      15067 non-null float64
surface_in_m2                 1077

In [18]:
#Al tener tantos registros, podemos darnos el lujo de borrar los 39 registros de propiedades que no tienen el barrio
#para homogeineizar el df.
properties.dropna(subset=['place_name'], inplace=True)
properties.dropna(subset=['lat'],inplace=True)
properties.dropna(subset=['lon'],inplace=True)
properties.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107671 entries, 16 to 196715
Data columns (total 17 columns):
created_on                    107671 non-null object
property_type                 107671 non-null object
place_name                    107671 non-null object
state_name                    107671 non-null object
lat-lon                       107671 non-null object
lat                           107671 non-null float64
lon                           107671 non-null float64
price                         107671 non-null float64
currency                      107667 non-null object
price_aprox_local_currency    107671 non-null float64
price_aprox_usd               107671 non-null float64
price_usd_per_m2              86351 non-null float64
price_per_m2                  103822 non-null float64
floor                         12034 non-null float64
rooms                         70035 non-null float64
expenses                      15065 non-null float64
surface_in_m2                 1076

Terminamos la limpieza y nos quedo un dataframe con 107671 registros, es decir, limpiamos aproximadamente el 20% de las publicaciones.

In [19]:
predict = pd.read_csv('properati_dataset_testing_noprice.csv', low_memory=False)

Unificamos la superficie

In [20]:
predict['surface_in_m2'] = predict[['surface_covered_in_m2','surface_total_in_m2']].apply(lambda x: obtener_superficie_final(x[0],x[1]), axis = 1)
predict = predict.drop(predict[['surface_covered_in_m2','surface_total_in_m2']],axis=1)

Y ordenemos las columnas para ayudar a su posterior estudio, eliminando las que no usemos y poniendolas en el mismo orden que en properties.

In [21]:
columnas = ['created_on', 'property_type', 'place_name', 'state_name', 'lat', 'lon','floor', 'rooms', 'expenses', 'surface_in_m2', 'id']
predict = predict[columnas]

Ahora unifiquemos el precio final

In [22]:
false = (properties['price'] != properties['price_aprox_usd'])
(properties.loc[false,:]['currency'] == 'ARS').value_counts()
#Se ve que para todo valor en el que price y price_aprox_usd no coinciden es porque esta expresado en pesos Argentinos,
#por lo que nos vamos a quedar siempre con price_aprox_usd.

True     4307
False       6
Name: currency, dtype: int64

In [23]:
properties = properties[['created_on','property_type', 'place_name', 'state_name', 'lat', 'lon', 'floor', 'rooms', 'expenses', 'surface_in_m2', 'price_aprox_usd', 'price_usd_per_m2']]

In [24]:
properties.info() 
predict.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107671 entries, 16 to 196715
Data columns (total 12 columns):
created_on          107671 non-null object
property_type       107671 non-null object
place_name          107671 non-null object
state_name          107671 non-null object
lat                 107671 non-null float64
lon                 107671 non-null float64
floor               12034 non-null float64
rooms               70035 non-null float64
expenses            15065 non-null float64
surface_in_m2       107671 non-null float64
price_aprox_usd     107671 non-null float64
price_usd_per_m2    86351 non-null float64
dtypes: float64(8), object(4)
memory usage: 10.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 11 columns):
created_on       14166 non-null object
property_type    14166 non-null object
place_name       14166 non-null object
state_name       14166 non-null object
lat              10487 non-null float64
lon       

# Empieza el TP2

Pasamos fecha a entero

In [25]:
def fecha_string_a_numero(fecha):
    return int(fecha.replace('-',''))

properties['created_on'] = properties['created_on'].apply(fecha_string_a_numero)
predict['created_on'] = predict['created_on'].apply(fecha_string_a_numero)

Pasamos a entero las expensas

In [26]:
def expensas_to_int(exp):
    try:
        aux = int(''.join(c for c in str(exp) if c.isdigit()));
    except:
        aux = 0
    return aux
        
        
properties['expenses'] = properties['expenses'].apply(expensas_to_int)
predict['expenses'] = predict['expenses'].apply(expensas_to_int)

Aquellos registros que tengan en el campo lon y/o lat un valor NaN se asigna el promedio del barrio

In [27]:
#Primero vemos que hay 3 propiedades de Terralagos que es en la localidad de Canning. En el dataset no tenemos datos 
# registros de ahi, por lo que cambiamos el place_name a Canning. 
#Tambien hay una propiedad de Buenos Aires Interior que la ubicaremos en Canning Tambien porque no tenemos ningún indicio
#de dónde puede ser.
properties.loc[properties.place_name.str.contains('San Mart'),['place_name']] = 'San Martín'

properties.loc[properties['place_name'] == 'Manzone',['place_name']] = 'Pilar'
properties.loc[properties['place_name'] == 'Villa Granaderos De San Martin',['place_name']] = 'San Martín'
properties.loc[properties['place_name'] == 'Villa Coronel Zapiola',['place_name']] = 'San Martín'
properties.loc[properties['place_name'] == 'Área de Promoción El Triángulo',['place_name']] = 'Malvinas Argentinas'
properties.loc[properties['place_name'] == 'Puerto Paraná',['place_name']] = 'Escobar'
properties.loc[properties['place_name'] == 'La Capilla',['place_name']] = 'Florencio Varela'
properties.loc[properties['place_name'] == 'La Honorata',['place_name']] = 'Pilar'
properties.loc[properties['place_name'] == 'Club de Campo Las Perdices',['place_name']] = 'Virrey del Pino'
properties.loc[properties['place_name'] == 'Terralagos',['place_name']] = 'Canning'
properties.loc[properties['place_name'] == 'El Viejo Vivero',['place_name']] = 'Don Torcuato'



In [28]:
promedio_lat_lon = properties[['place_name','lat','lon']].groupby('place_name').agg(np.mean).reset_index()
promedio_lat_lon.columns = ['place_name','lat_prom','lon_prom']


lat_menor = properties.lat < -37
lat_mayor = properties.lat > -33
lon_menor = properties.lon < -60
lon_mayor = properties.lon > -56
lat_nan = np.isnan(properties['lat'])
lon_nan = np.isnan(properties['lon'])

conjunto = np.logical_or(np.logical_or(np.logical_or(lat_menor,lat_mayor),np.logical_or(lon_menor,lon_mayor)),\
                         np.logical_or(lat_nan,lon_nan))


properties_without_latlon = properties.loc[conjunto ,['lat','lon','place_name']].reset_index()
properties_without_latlon = properties_without_latlon.merge(promedio_lat_lon, how='left', left_on = 'place_name', right_on = 'place_name')
properties_without_latlon = properties_without_latlon.loc[:,['index','lat_prom','lon_prom']].set_index('index')
properties.loc[conjunto ,['lat','lon']] = properties_without_latlon[['lat_prom','lon_prom']].values

#Elimino un registro que estaba en una localidad con un solo registro y sin lat lon.
properties = properties[np.isfinite(properties['lon'])]

Hagamos lo mismo con el dataset que tenemos que predecir

In [29]:
#Primero vemos que hay 3 propiedades de Terralagos que es en la localidad de Canning. En el dataset no tenemos datos 
# registros de ahi, por lo que cambiamos el place_name a Canning. 
#Tambien hay una propiedad de Buenos Aires Interior que la ubicaremos en Canning Tambien porque no tenemos ningún indicio
#de dónde puede ser.
predict.loc[predict.place_name.str.contains('San Mart'),['place_name']] = 'San Martín'


predict.loc[predict['place_name'] == 'Terralagos',['place_name']] = 'Canning'
predict.loc[predict['place_name'] == 'Buenos Aires Interior',['place_name']] = 'Canning'



In [30]:
lat_menor = predict.lat < -37
lat_mayor = predict.lat > -33
lon_menor = predict.lon < -60
lon_mayor = predict.lon > -56
lat_nan = np.isnan(predict['lat'])
lon_nan = np.isnan(predict['lon'])

conjunto = np.logical_or(np.logical_or(np.logical_or(lat_menor,lat_mayor),np.logical_or(lon_menor,lon_mayor)),\
                         np.logical_or(lat_nan,lon_nan))

predict_without_latlon = predict.loc[conjunto ,['lat','lon','place_name']].reset_index()
predict_without_latlon = predict_without_latlon.merge(promedio_lat_lon, how='left', left_on = 'place_name', right_on = 'place_name')
predict_without_latlon = predict_without_latlon.loc[:,['index','lat_prom','lon_prom']].set_index('index')

predict.loc[conjunto ,['lat','lon']] = predict_without_latlon[['lat_prom','lon_prom']].values

Vamos a pasar los tipos de propiedad del set de trabajo a numero, para esto definimos que:
    
    -Si el tipo de propiedad es una casa se representa con el valor 1
    
    -Si el tipo de propiedad es un departamento se representa con el valor 2
    
    -Si el tipo de propiedad es un ph se representa con el valor 3
    
    -Si el tipo de propiedad es una oficina se representa con el valor 4




In [31]:
def traducir_property_type(tipo):
    aux=tipo
    if (tipo == "apartment" or tipo == "departamento"):
        aux=int(2)
    elif (tipo == "house" or tipo == "casa"):
        aux=int(1)
    elif (tipo == "store"):
        aux=int(4)
    elif (tipo == "PH" or tipo == "ph"):
        aux=int(3)
    return aux

properties['property_type'] = properties['property_type'].apply(traducir_property_type)
predict['property_type'] = predict['property_type'].apply(traducir_property_type)

In [32]:
def traducir_piso(floor):
    try:
        floor = int(floor)
    except:
        floor = 0
    return floor

properties['floor'] = properties['floor'].apply(traducir_piso)
predict['floor'] = predict['floor'].apply(traducir_piso)


In [33]:
def traducir_rooms(rooms):
    try:
        rooms = int(rooms)
    except:
        rooms = 0
    return rooms

properties['rooms'] = properties['rooms'].apply(traducir_rooms)
predict['rooms'] = predict['rooms'].apply(traducir_rooms)

predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 11 columns):
created_on       14166 non-null int64
property_type    14166 non-null int64
place_name       14166 non-null object
state_name       14166 non-null object
lat              14166 non-null float64
lon              14166 non-null float64
floor            14166 non-null int64
rooms            14166 non-null int64
expenses         14166 non-null int64
surface_in_m2    14166 non-null float64
id               14166 non-null int64
dtypes: float64(3), int64(6), object(2)
memory usage: 1.2+ MB


In [34]:
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsRegressor

def knn_por_precio_total(properties, predict, k_knn, columnas):
    x = properties.loc[:,columnas]
    y = properties.loc[:,['price_aprox_usd']]

    neigh = KNeighborsRegressor(n_neighbors=k_knn, weights='distance', n_jobs=-1)
    neigh.fit(x,y)
    predict.loc[:,['prediccion']] = neigh.predict(predict.loc[:, columnas])
    return predict


def normalizar_con_peso(train, test, columnas, pesos):
    if len(columnas) != len(pesos):
        raise ValueError()
       
    for i in range(len(columnas)):
        promedio = train[[columnas[i]]].mean()
        desvio = train[[columnas[i]]].std() 
        train[[columnas[i]]] = (train[[columnas[i]]] - promedio) / desvio * pesos[i]
        test[[columnas[i]]] = (test[[columnas[i]]] - promedio) / desvio * pesos[i]
    return train,test



def calcular_error(test):
    error = 0
    for elem in test:
        error += abs(elem[0]-elem[1])
    return error/len(test)
    

### Acá probamos darle diferente importancia a las columnas que le pasamos

In [35]:
def crear_ponderaciones(lista_tipoPropiedad, lista_latlon, lista_surface):
    l = []
    for tipo in lista_tipoPropiedad:
        for latlon in lista_latlon:
            for surface in lista_surface:
                l.append([tipo,latlon,latlon,surface])
    return l

### Funcion de redondeo

In [36]:
def redondear(price):
    aux = price
    aux2= aux%1000
    if (aux2>499):
        price=price-aux2+1000
    elif (aux2<=499):
        price=price-aux2
    return price
    
    

## Predecimos las propiedades

In [37]:
from sklearn.model_selection import train_test_split
from time import time
ti = time()

valores_knn = [8,10,11,15,17,20,25,30,40,50,60,75,80,90,100]
resultados = []
todas_columnas = ['created_on','property_type','lat','lon','floor','rooms','surface_in_m2']
columnas_mejorado = ['property_type','lat','lon','surface_in_m2']

predict['prediccion'] = np.abs(0)

ponderaciones = crear_ponderaciones([4,6,7,8,10,12,14],\
                                    [7,9,11,13,15,17,18,20],\
                                    [1])

for lista_puntajes in ponderaciones:
    for k_knn in valores_knn:         
        properties_norm, predict_norm = normalizar_con_peso(properties.copy(),predict.copy(),columnas_mejorado,lista_puntajes)       
        predict_final = knn_por_precio_total(properties_norm,predict_norm,k_knn,columnas_mejorado)
        predict['prediccion'] += predict_final['prediccion']
        print("Termino con K=",k_knn)
    print(lista_puntajes)   

divisor = len(valores_knn) * len(ponderaciones)        
        
predict['price_usd2'] = predict['prediccion'].apply(lambda x: x/divisor)

predict['price_usd'] = predict['price_usd2'].apply(lambda x: redondear(x))

predict[['id','price_usd']].to_csv(path_or_buf = 'prediccion.csv', index = False)

print("Tiempo utilizado: ", time()-ti)

Termino con K= 8
Termino con K= 10
Termino con K= 11
Termino con K= 15
Termino con K= 17
Termino con K= 20
Termino con K= 25
Termino con K= 30
Termino con K= 40
Termino con K= 50
Termino con K= 60
Termino con K= 75
Termino con K= 80
Termino con K= 90
Termino con K= 100
[4, 7, 7, 1]
Termino con K= 8
Termino con K= 10
Termino con K= 11
Termino con K= 15
Termino con K= 17
Termino con K= 20
Termino con K= 25
Termino con K= 30
Termino con K= 40
Termino con K= 50
Termino con K= 60
Termino con K= 75
Termino con K= 80
Termino con K= 90
Termino con K= 100
[4, 9, 9, 1]
Termino con K= 8
Termino con K= 10
Termino con K= 11
Termino con K= 15
Termino con K= 17
Termino con K= 20
Termino con K= 25
Termino con K= 30
Termino con K= 40
Termino con K= 50
Termino con K= 60
Termino con K= 75
Termino con K= 80
Termino con K= 90
Termino con K= 100
[4, 11, 11, 1]
Termino con K= 8
Termino con K= 10
Termino con K= 11
Termino con K= 15
Termino con K= 17
Termino con K= 20
Termino con K= 25
Termino con K= 30
Termin

Termino con K= 90
Termino con K= 100
[8, 15, 15, 1]
Termino con K= 8
Termino con K= 10
Termino con K= 11
Termino con K= 15
Termino con K= 17
Termino con K= 20
Termino con K= 25
Termino con K= 30
Termino con K= 40
Termino con K= 50
Termino con K= 60
Termino con K= 75
Termino con K= 80
Termino con K= 90
Termino con K= 100
[8, 17, 17, 1]
Termino con K= 8
Termino con K= 10
Termino con K= 11
Termino con K= 15
Termino con K= 17
Termino con K= 20
Termino con K= 25
Termino con K= 30
Termino con K= 40
Termino con K= 50
Termino con K= 60
Termino con K= 75
Termino con K= 80
Termino con K= 90
Termino con K= 100
[8, 18, 18, 1]
Termino con K= 8
Termino con K= 10
Termino con K= 11
Termino con K= 15
Termino con K= 17
Termino con K= 20
Termino con K= 25
Termino con K= 30
Termino con K= 40
Termino con K= 50
Termino con K= 60
Termino con K= 75
Termino con K= 80
Termino con K= 90
Termino con K= 100
[8, 20, 20, 1]
Termino con K= 8
Termino con K= 10
Termino con K= 11
Termino con K= 15
Termino con K= 17
Term